In [2]:
!pip install quantstats

In [8]:
import os
import pandas as pd
import quantstats as qs

def metrics(data):
    """
    Calculate metrics for each asset in the DataFrame.

    Parameters:
    data (DataFrame): Input DataFrame containing historical price data.

    Returns:
    DataFrame: DataFrame with calculated metrics for each asset.
    """
    grouped_data = data.groupby("asset_name")

    data["ratio_sharpe"] = data["asset_name"].map(grouped_data.apply(lambda x: qs.stats.sharpe(x["Close"])))
    data["ratio_sortino"] = data["asset_name"].map(grouped_data.apply(lambda x: qs.stats.sortino(x["Close"])))
    data["ratio_win_loss"] = data["asset_name"].map(grouped_data.apply(lambda x: qs.stats.win_loss_ratio(x["Close"])))
    data["percentage_drawdown"] = data["asset_name"].map(grouped_data.apply(lambda x: qs.stats.max_drawdown(x["Close"])))

    data["Date"] = pd.to_datetime(data["Date"])
    data_1_d = data[data["Date"] >= data["Date"].max() - pd.DateOffset(days=1)]
    data_1_m = data[data["Date"] >= data["Date"].max() - pd.DateOffset(months=1)]
    data_3_m = data[data["Date"] >= data["Date"].max() - pd.DateOffset(months=3)]
    data_1_y = data[data["Date"] >= data["Date"].max() - pd.DateOffset(years=1)]

    data_1_d.set_index("Date", inplace=True)
    data_1_m.set_index("Date", inplace=True)
    data_3_m.set_index("Date", inplace=True)
    data_1_y.set_index("Date", inplace=True)

    data_1_d_grouped = data_1_d.groupby("asset_name")
    data_1_m_grouped = data_1_m.groupby("asset_name")
    data_3_m_grouped = data_3_m.groupby("asset_name")
    data_1_y_grouped = data_1_y.groupby("asset_name")

    data["percentage_1_d_cagr"] = data["asset_name"].map(data_1_d_grouped.apply(lambda x: qs.stats.cagr(x["Close"])))
    data["percentage_1_m_cagr"] = data["asset_name"].map(data_1_m_grouped.apply(lambda x: qs.stats.cagr(x["Close"])))
    data["percentage_3_m_cagr"] = data["asset_name"].map(data_3_m_grouped.apply(lambda x: qs.stats.cagr(x["Close"])))
    data["percentage_1_y_cagr"] = data["asset_name"].map(data_1_y_grouped.apply(lambda x: qs.stats.cagr(x["Close"])))

    data["percentage_1_d_volatility"] = data["asset_name"].map(data_1_d_grouped.apply(lambda x: qs.stats.volatility(x["Close"])))
    data["percentage_1_m_volatility"] = data["asset_name"].map(data_1_m_grouped.apply(lambda x: qs.stats.volatility(x["Close"])))
    data["percentage_3_m_volatility"] = data["asset_name"].map(data_3_m_grouped.apply(lambda x: qs.stats.volatility(x["Close"])))
    data["percentage_1_y_volatility"] = data["asset_name"].map(data_1_y_grouped.apply(lambda x: qs.stats.volatility(x["Close"])))

    return data

# Function to determine action (Go Long, Hold Cash)
def determine_action(row):
    if pd.isnull(row['10_month_MA']):
        return 'Cash'
    elif row['Close'] > row['10_month_MA']:
        return 'Go Long'
    else:
        return 'Hold Cash'

# Function to map asset names to categories
def map_asset_category(asset_name):
    if asset_name in ['SPY', 'MTUM', 'IWN', 'EFA', 'EEM', 'XHB', 'XLB', 'XLE', 'XLY', 'XLK', 'XLV', 'XLI', 'XLU', 'XLP', 'XLF', 'XLC', 'XLRE']:
        return "ETF"
    elif asset_name in ['DBC', 'GLD']:
        return "Gold"
    elif asset_name in ['BIL', 'IEF', 'BWX', 'LQD', 'TLT']:
        return "Treasury"
    elif asset_name == 'VNQ':
        return "REIT"
    else:
        return "Stock"

# Path to the folder containing CSV files
folder_path = r"C:\Users\DELL\Desktop\Projects\Codeshastra X\Data\Broad Indices"

# Get a list of all CSV files in the folder
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# Iterate through each CSV file
dfs = []
for file in csv_files:
    filename = os.path.splitext(file)[0]
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    df['asset_name'] = filename
    data = metrics(df)
    dfs.append(data)

# Combine all DataFrames
combined_df = pd.concat(dfs, ignore_index=True)

# Percentage allocations for each asset
percentage_allocations = {
    'SPY': 5,
    'MTUM': 10,
    'IWN': 5,
    'EFA': 10,
    'EEM': 10,
    'IEF': 5,
    'BWX': 5,
    'LQD': 5,
    'TLT': 5,
    'DBC': 10,
    'GLD': 10,
    'VNQ': 20
}

# Map percentage allocations to asset names
combined_df['percentage_allocation'] = combined_df['asset_name'].map(percentage_allocations)

# Drop rows with NaN allocations
combined_df.dropna(subset=['percentage_allocation'], inplace=True)

# Calculate 10-month moving average for each asset
combined_df['10_month_MA'] = combined_df.groupby('asset_name')['Close'].transform(lambda x: x.rolling(window=10).mean())


# Apply the function to each row to determine the action
combined_df['Action'] = combined_df.apply(determine_action, axis=1)

# Filter only the last trading day of each month
last_day_of_month = combined_df.groupby([combined_df['Date'].dt.year, combined_df['Date'].dt.month])['Date'].max()
combined_df['LastDayOfMonth'] = combined_df['Date'].isin(last_day_of_month)
combined_df.loc[combined_df['LastDayOfMonth'], 'Action'] = 'Hold Cash'
combined_df.drop('LastDayOfMonth', axis=1, inplace=True)

# Drop unnecessary columns
combined_df.drop(['10_month_MA'], axis=1, inplace=True)

# Filter rows with actions other than 'Hold Cash' or 'Cash'
combined_df = combined_df[(combined_df['Action'] != 'Hold Cash') & (combined_df['Action'] != 'Cash')]

# Apply the function to create the 'asset_category' column
combined_df['asset_category'] = combined_df['asset_name'].apply(map_asset_category)
combined_df.drop('Action', axis = 1, inplace = True)
# Display the resulting DataFrame
combined_df


,Date,Open,High,Low,Close,Adj Close,Volume,asset_name,ratio_sharpe,ratio_sortino,...,percentage_1_d_cagr,percentage_1_m_cagr,percentage_3_m_cagr,percentage_1_y_cagr,percentage_1_d_volatility,percentage_1_m_volatility,percentage_3_m_volatility,percentage_1_y_volatility,percentage_allocation,asset_category
4247,2007-10-24,26.350000,26.389999,26.264999,26.375000,21.149965,146200,BWX,-0.061105,-0.085152,...,-0.432164,0.011799,-0.109376,-0.026365,0.02518,0.052630,0.074750,0.091849,5.0,Treasury
4248,2007-10-25,26.485001,26.485001,26.424999,26.445000,21.206106,60000,BWX,-0.061105,-0.085152,...,-0.432164,0.011799,-0.109376,-0.026365,0.02518,0.052630,0.074750,0.091849,5.0,Treasury
4249,2007-10-26,26.400000,26.535000,26.400000,26.530001,21.274263,76200,BWX,-0.061105,-0.085152,...,-0.432164,0.011799,-0.109376,-0.026365,0.02518,0.052630,0.074750,0.091849,5.0,Treasury
4250,2007-10-29,26.549999,26.605000,26.525000,26.590000,21.322382,377600,BWX,-0.061105,-0.085152,...,-0.432164,0.011799,-0.109376,-0.026365,0.02518,0.052630,0.074750,0.091849,5.0,Treasury
4251,2007-10-30,26.549999,26.575001,26.540001,26.549999,21.290298,91600,BWX,-0.061105,-0.085152,...,-0.432164,0.011799,-0.109376,-0.026365,0.02518,0.052630,0.074750,0.091849,5.0,Treasury
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66591,2024-03-08,87.500000,88.029999,87.300003,87.730003,87.730003,4546400,VNQ,0.243817,0.347044,...,4.952548,0.182223,-0.088422,0.068837,0.07974,0.161601,0.170656,0.184994,20.0,REIT
66592,2024-03-11,87.540001,88.070000,86.940002,87.309998,87.309998,3467200,VNQ,0.243817,0.347044,...,4.952548,0.182223,-0.088422,0.068837,0.07974,0.161601,0.170656,0.184994,20.0,REIT
66593,2024-03-12,87.150002,87.480003,86.370003,87.120003,87.120003,3309000,VNQ,0.243817,0.347044,...,4.952548,0.182223,-0.088422,0.068837,0.07974,0.161601,0.170656,0.184994,20.0,REIT
66600,2024-03-21,86.129997,86.750000,85.860001,86.379997,86.379997,3753200,VNQ,0.243817,0.347044,...,4.952548,0.182223,-0.088422,0.068837,0.07974,0.161601,0.170656,0.184994,20.0,REIT


In [9]:
from flask import Flask, request, jsonify
import os
import pandas as pd
import quantstats as qs
import pyodbc
app = Flask(__name__)

# SQL Server connection parameters
driver = "{ODBC Driver 17 for SQL Server}"
server = "localhost\\SQLEXPRESS"
database = "your_database_name"
trusted_connection = "yes"

# Construct connection string
DB_CONNECTION_STRING = f"DRIVER={driver};SERVER={server};DATABASE={database};TRUSTED_CONNECTION={trusted_connection};"

def connect_to_database():
    return pyodbc.connect(DB_CONNECTION_STRING)
def metrics(data):
    """
    Calculate metrics for each asset in the DataFrame.

    Parameters:
    data (DataFrame): Input DataFrame containing historical price data.

    Returns:
    DataFrame: DataFrame with calculated metrics for each asset.
    """
    grouped_data = data.groupby("asset_name")

    data["ratio_sharpe"] = data["asset_name"].map(grouped_data.apply(lambda x: qs.stats.sharpe(x["Close"])))
    data["ratio_sortino"] = data["asset_name"].map(grouped_data.apply(lambda x: qs.stats.sortino(x["Close"])))
    data["ratio_win_loss"] = data["asset_name"].map(grouped_data.apply(lambda x: qs.stats.win_loss_ratio(x["Close"])))
    data["percentage_drawdown"] = data["asset_name"].map(grouped_data.apply(lambda x: qs.stats.max_drawdown(x["Close"])))

    data["Date"] = pd.to_datetime(data["Date"])
    data_1_d = data[data["Date"] >= data["Date"].max() - pd.DateOffset(days=1)]
    data_1_m = data[data["Date"] >= data["Date"].max() - pd.DateOffset(months=1)]
    data_3_m = data[data["Date"] >= data["Date"].max() - pd.DateOffset(months=3)]
    data_1_y = data[data["Date"] >= data["Date"].max() - pd.DateOffset(years=1)]

    data_1_d.set_index("Date", inplace=True)
    data_1_m.set_index("Date", inplace=True)
    data_3_m.set_index("Date", inplace=True)
    data_1_y.set_index("Date", inplace=True)

    data_1_d_grouped = data_1_d.groupby("asset_name")
    data_1_m_grouped = data_1_m.groupby("asset_name")
    data_3_m_grouped = data_3_m.groupby("asset_name")
    data_1_y_grouped = data_1_y.groupby("asset_name")

    data["percentage_1_d_cagr"] = data["asset_name"].map(data_1_d_grouped.apply(lambda x: qs.stats.cagr(x["Close"])))
    data["percentage_1_m_cagr"] = data["asset_name"].map(data_1_m_grouped.apply(lambda x: qs.stats.cagr(x["Close"])))
    data["percentage_3_m_cagr"] = data["asset_name"].map(data_3_m_grouped.apply(lambda x: qs.stats.cagr(x["Close"])))
    data["percentage_1_y_cagr"] = data["asset_name"].map(data_1_y_grouped.apply(lambda x: qs.stats.cagr(x["Close"])))

    data["percentage_1_d_volatility"] = data["asset_name"].map(data_1_d_grouped.apply(lambda x: qs.stats.volatility(x["Close"])))
    data["percentage_1_m_volatility"] = data["asset_name"].map(data_1_m_grouped.apply(lambda x: qs.stats.volatility(x["Close"])))
    data["percentage_3_m_volatility"] = data["asset_name"].map(data_3_m_grouped.apply(lambda x: qs.stats.volatility(x["Close"])))
    data["percentage_1_y_volatility"] = data["asset_name"].map(data_1_y_grouped.apply(lambda x: qs.stats.volatility(x["Close"])))

    return data

def determine_action(row):
    if pd.isnull(row['10_month_MA']):
        return 'Cash'
    elif row['Close'] > row['10_month_MA']:
        return 'Go Long'
    else:
        return 'Hold Cash'

def map_asset_category(asset_name):
    if asset_name in ['SPY', 'MTUM', 'IWN', 'EFA', 'EEM', 'XHB', 'XLB', 'XLE', 'XLY', 'XLK', 'XLV', 'XLI', 'XLU', 'XLP', 'XLF', 'XLC', 'XLRE']:
        return "ETF"
    elif asset_name in ['DBC', 'GLD']:
        return "Gold"
    elif asset_name in ['BIL', 'IEF', 'BWX', 'LQD', 'TLT']:
        return "Treasury"
    elif asset_name == 'VNQ':
        return "REIT"
    else:
        return "Stock"

@app.route('/calculate_metrics', methods=['POST'])
def calculate_metrics():
    conn = connect_to_database()
    cursor = conn.cursor()
    portfolio_name = request.json.get('portfolio_name')
    if not portfolio_name:
        return jsonify({'error': 'Portfolio name is required'}), 400
    
    folder_path = r"C:\Users\DELL\Desktop\Projects\Codeshastra X\Data\Broad Indices"
    csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

    dfs = []
    for file in csv_files:
        filename = os.path.splitext(file)[0]
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        df['asset_name'] = filename
        data = metrics(df)
        dfs.append(data)

    combined_df = pd.concat(dfs, ignore_index=True)

    percentage_allocations = {
        'SPY': 5,
        'MTUM': 10,
        'IWN': 5,
        'EFA': 10,
        'EEM': 10,
        'IEF': 5,
        'BWX': 5,
        'LQD': 5,
        'TLT': 5,
        'DBC': 10,
        'GLD': 10,
        'VNQ': 20
    }

    combined_df['percentage_allocation'] = combined_df['asset_name'].map(percentage_allocations)
    combined_df.dropna(subset=['percentage_allocation'], inplace=True)
    combined_df['10_month_MA'] = combined_df.groupby('asset_name')['Close'].transform(lambda x: x.rolling(window=10).mean())
    combined_df['Action'] = combined_df.apply(determine_action, axis=1)
    last_day_of_month = combined_df.groupby([combined_df['Date'].dt.year, combined_df['Date'].dt.month])['Date'].max()
    combined_df['LastDayOfMonth'] = combined_df['Date'].isin(last_day_of_month)
    combined_df.loc[combined_df['LastDayOfMonth'], 'Action'] = 'Hold Cash'
    combined_df.drop('LastDayOfMonth', axis=1, inplace=True)
    combined_df.drop(['10_month_MA'], axis=1, inplace=True)
    combined_df = combined_df[(combined_df['Action'] != 'Hold Cash') & (combined_df['Action'] != 'Cash')]
    combined_df['asset_category'] = combined_df['asset_name'].apply(map_asset_category)
    combined_df.drop('Action', axis=1, inplace=True)
    
    current_date = datetime.now().strftime('%Y-%m-%d')
    combined_df['current_date'] = current_date
    combined_df['asset_creation_date'] = current_date

    # Update or insert combined_df into portfolio_details table
    for index, row in combined_df.iterrows():
        cursor.execute("SELECT * FROM portfolio_details WHERE portfolio_name = ? AND asset_name = ? AND current_date = ?", (portfolio_name, row['asset_name'], current_date))
        existing_data = cursor.fetchone()
        if existing_data:
            cursor.execute("UPDATE portfolio_details SET asset_creation_date = ?, other_column = ? WHERE portfolio_name = ? AND asset_name = ? AND current_date = ?", (current_date, row['other_column'], portfolio_name, row['asset_name'], current_date))
        else:
            cursor.execute("INSERT INTO portfolio_details (portfolio_name, asset_name, current_date, asset_creation_date, other_column) VALUES (?, ?, ?, ?, ?)", (portfolio_name, row['asset_name'], current_date, current_date, row['other_column']))

    total_allocation = combined_df["percentage_allocation"].sum()
    cash_allocation = 100 - total_allocation
    
    # Delete data from portfolio details table
    cursor.execute("DELETE FROM portfolio_details WHERE portfolio_name = ? AND current_date = (SELECT MAX(current_date) FROM portfolio_details WHERE portfolio_name = ?)", (portfolio_name, portfolio_name))

    # Delete data from portfolio_performance table
    cursor.execute("DELETE FROM portfolio_performance WHERE portfolio_name = ? AND current_date = (SELECT MAX(current_date) FROM portfolio_performance WHERE portfolio_name = ?)", (portfolio_name, portfolio_name))

    # Update percentage_cash value to 50 in latest_allocation table
    cursor.execute(f"UPDATE latest_allocation SET percentage_allocation_cash = {cash_allocation} WHERE portfolio_name = ?", (portfolio_name,))
            
    
    return

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\DELL\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
